In [ ]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

In [ ]:
# Necessary Imports

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv
from random import uniform
import re

In [ ]:
df_name = pd.read_csv("product_names.csv")

In [ ]:
# Chromium Headless Options

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--remote-debugging-port=9222")

driver = webdriver.Chrome(options=chrome_options)
pattern = (
    r"https://www\.intel\.com/content/www/us/en/products/sku/.+/specifications\.html"
)


names_dict = {"product_name": [], "product_fullName": []}

with open("product_names.csv", mode="r") as file:
    csv_reader = csv.DictReader(file)
    for row in csv_reader:
        baseUrl = row["URL"]
        google_search = "https://www.google.com/search?q=" + baseUrl

        driver.get(google_search)
        content_div = driver.find_element(By.ID, "rcnt").get_attribute("outerHTML")
        page_html = BeautifulSoup(content_div, "html.parser")
        a_tags = page_html.find_all("a")

        for tag in a_tags:
            url = tag.get("href")
            if url is None:
                continue
            match = re.match(pattern, url)
            if match:
                driver.get(url)
                processor_html = driver.find_element(
                    By.ID, "specs-1-0-0"
                ).get_attribute("outerHTML")
                processor_bs4 = BeautifulSoup(processor_html, "html.parser")
                a = processor_bs4.find_all("div", class_="col-xs-6 col-lg-6 tech-data")
                product_name = a[3].get_text().strip()

                print(product_name)

                names_dict["product_name"].append(product_name)
                names_dict["product_fullName"].append(baseUrl)

        driver.implicitly_wait(5)


df = pd.DataFrame(names_dict)
while True:
    if os.path.isfile(f"./product_names/{baseUrl}.csv"):
        baseUrl = baseUrl + "-" + str(int(uniform(1, 100)))
    else:
        break
csv_filename = f"./product_names/{baseUrl}.csv"
df.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")